# Quick Start Guide

This tutorial shows how to quickly get started using *scarlet* to model an hyperspectral image cube. For a more in-depth introduction to *scarlet*, read the [Core Concepts](concepts.ipynb) or the [API Documentation](api/index.rst).

In [ ]:
# Import Packages and setup
import logging
logger = logging.getLogger('scarlet')
logger.setLevel(logging.DEBUG)
logger = logging.getLogger("proxmin")
logger.setLevel(logging.DEBUG)


import autograd.numpy as np
import scarlet
import scarlet.display

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
# use a better colormap and don't interpolate the pixels
matplotlib.rc('image', cmap='inferno')
matplotlib.rc('image', interpolation='none')

## Load and display data

We load an example data set (here an image cube with 5 bands) *and* a detection catalog.
If such a catalog is not available packages like [SEP](http://sep.readthedocs.io/) and [photutils](https://photutils.readthedocs.io/en/stable/) will happily generate one, but for this example we use part of the detection catalog generated by the [LSST DM stack](https://github.com/lsst).

In [ ]:
# Load the sample images
data = np.load("../data/hsc_cosmos_35.npz")
images = data["images"]
filters = data["filters"]
catalog = data["catalog"]
weights = 1/data["variance"]
psfs = scarlet.PSF(data["psfs"])

### Display a raw image cube
This is an example of how to display an RGB image from an image cube of multiband data. In this case the image uses a $sinh^{-1}$ function to normalize the flux in each filter consistently to create an RGB image.

In [ ]:
from astropy.visualization.lupton_rgb import AsinhMapping, LinearMapping

stretch = 0.1
Q = 10
norm = AsinhMapping(minimum=0, stretch=stretch, Q=Q)
img_rgb = scarlet.display.img_to_rgb(images, norm=norm)
plt.imshow(img_rgb)

# Mark all of the sources from the detection cataog
for k, src in enumerate(catalog):
    plt.text(src["x"], src["y"], str(k), color="red")

## Define model frame and observation

A `Frame` in *scarlet* is a description of the hyperspectral cube of the model or the observations. Think of it as the metadata, what aspects of the sky are described here. At the least, a `Frame` holds the `shape` of the cube, for which we use the convention `(C, Ny, Nx)` for the number of elements in 3 dimensions: `C` for the number of bands/channels and `Ny, Nx` for the number of pixels at every channel.

An `Observation` combines a `Frame` with several data units, similar to header-data arrangement in FITS files. In addition the the actual science image cube, you can and often must provide weights for all elements in the data cube, an image cube of the PSF model (one image for all or one for each channel), an `astropy.WCS` structure to translate from pixel to sky coordinates, and labels for all channels. The reason for specifying them is to enable the code to internally map from the model frame, in which you seek to fit a model, to the observed data frame.

In this example, we assume that bands and pixel locations are identical between the model and the observation. Because we have ground-based images with different PSFs in each band, we need to provide a reference PSF for the model. We simply choose a minimal Gaussian PSF that is barely well sampled as our reference kernel:

In [ ]:
from functools import partial
model_psf = scarlet.PSF(partial(scarlet.psf.gaussian, sigma=.8), shape=(None, 8, 8))

With this we can fully specify the `Frame` and `Observation`:

In [ ]:
model_frame = scarlet.Frame(
    images.shape,
    psf=model_psf,
    channels=filters)

observation = scarlet.Observation(
    images, 
    psf=psfs, 
    weights=weights, 
    channels=filters).match(model_frame)

The last command calls the `match` method to compute e.g. PSF difference kernel and filter transformations.

We generally recommend this pattern:
1. define model frame
2. construct observation
3. match it to the model frame

Steps 2 and 3 are combined above using a fluent pattern.

## Initialize sources

You now need to define sources that are going to be fit. The full model, which we will call `Blend`, is a collection of those sources. We provide several pre-built source types:

* `RandomSource` fit per-band amplitude and non-parametric morphology starting from uniform random draws for both.
* `PointSource` fits centers and per-band amplitude using the observed PSF model.
* `ExtendedSource` fits per-band amplitude and a non-parametric morphology (which can be constrained to be symmetric and/or monotonic with respect to the center).
* `MultiComponentSource` splits an `ExtendedSource` into multiple components that are initially radially separated.

In our example, we assume *prior* knowledge that object 0 is a star, and object 1 should be modeled with two components. Everything else is assumed a single-component galaxy.

In [ ]:
sources = []
for k,src in enumerate(catalog):
    if k == 0:
        new_source = scarlet.PointSource(model_frame, (src['y'], src['x']), observation)
    elif k == 1:
        new_source = scarlet.MultiComponentSource(model_frame, (src['y'], src['x']), observation, symmetric=False, monotonic=True, thresh=5)
    else:
        new_source = scarlet.ExtendedSource(model_frame, (src['y'], src['x']), observation, symmetric=False, monotonic=True, thresh=5)
    sources.append(new_source)

## Create and fit model
The `Blend` class represent the sources as a tree and has the machinery to fit all of the sources to the given images. In this example the code is set to run for a maximum of 200 iterations, but will end early if the likelihood and all of the constraints converge.

In [ ]:
blend = scarlet.Blend(sources, observation)
%time blend.fit(200)
print("scarlet ran for {0} iterations to logL = {1}".format(len(blend.loss), -blend.loss[-1]))
plt.plot(-np.array(blend.loss))
plt.xlabel('Iteration')
plt.ylabel('log-Likelihood')

## Interact with Results

### View the full model
First we load the model for the entire scene, render it in the observation frame, and compute its residuals. We then show model and date with the same $sinh^{-1}$ stretch and the residuals with a linear stretch.

In [ ]:
# Load the model and calculate the residual
model = blend.get_model()
model_ = observation.render(model)
residual = images-model_
# Create RGB images
model_rgb = scarlet.display.img_to_rgb(model_, norm=norm)
residual_rgb = scarlet.display.img_to_rgb(residual)

# Show the data, model, and residual
fig = plt.figure(figsize=(15,5))
ax = [fig.add_subplot(1,3,n+1) for n in range(3)]
ax[0].imshow(img_rgb)
ax[0].set_title("Data")
ax[1].imshow(model_rgb)
ax[1].set_title("Model")
ax[2].imshow(residual_rgb)
ax[2].set_title("Residual")

for k,component in enumerate(blend):
    y,x = component.center
    ax[0].text(x, y, k, color="w")
    ax[1].text(x, y, k, color="w")
    ax[2].text(x, y, k, color="w")
plt.show()

### View the source models
We will now inspect the model for each source, in its original frame and in its observed frame. In this example, the two frames differ by an extra convolution from the minimal `model_psf` to the observed psfs.

In [ ]:
# Set the stretch based on the model
stretch = .3
Q = 10
norm = AsinhMapping(minimum=0, stretch=stretch, Q=Q)

for k,src in enumerate(sources):
    # Get the model for a single source
    model = src.get_model()
    model_ = observation.render(model)
    
    # Convert observation and models to RGB
    img_rgb = scarlet.display.img_to_rgb(images, norm=norm)
    model_rgb = scarlet.display.img_to_rgb(model, norm=norm)
    model_rgb_ = scarlet.display.img_to_rgb(model_, norm=norm)

    # Set the figure size
    ratio = src.frame.shape[2]/src.frame.shape[1]
    fig_height = 3*src.frame.shape[1]/20
    fig_width = max(2*fig_height*ratio,2)
    fig = plt.figure(figsize=(fig_width, fig_height))
    
    # Generate and show the figure
    ax = [fig.add_subplot(1,3,n+1) for n in range(3)]
    ax[0].imshow(img_rgb)
    ax[0].set_title("Data")
    ax[1].imshow(model_rgb_)
    ax[1].set_title("Observed model {0}".format(k))
    ax[2].imshow(model_rgb)
    ax[2].set_title("Model {0}".format(k))
    # Mark the source in the data image
    y,x = src.center
    ax[0].plot(x, y, "wx", mew=1, ms=10)
    ax[1].plot(x, y, "wx", mew=1, ms=10)
    ax[2].plot(x, y, "wx", mew=1, ms=10)
    plt.show()

We can see that the model of object 0 assumes the simple Gaussian shape of the model PSF, which is the internal representation of a point source. Source 1 uses the freedom of the 2-compoent model to represent a slightly redder core.

### Measure Fluxes

The color information in these plots stems from the per-band amplitude, which could be obtained as `source.sed`. However, it is more useful to compute per-band fluxes, which integrate over the morphology. The convention of these fluxes is given by the units and ordering of the original data cube. In the case of multi-component sources, the fluxes of all components are combined.

In [ ]:
print ("----------------- {}".format(filters))
for k, src in enumerate(sources):
    print ("Source {}, Fluxes: {}".format(k, scarlet.measure.flux(src)))

Other measurements (e.g. `centroid`) are also implemented. The measurement approach is also easily extendable. The source models are generated in the model frame (which is the best-fit representation of the full hyperspectral `Frame`), from which any measurement can directly be made without having to deal with noise, PSF convolution, overlapping sources, etc. 

### Save and Re-Use Model

To preserve the model for posterity, individual sources or lists of sources can be serialized with the `pickle` library:

In [ ]:
import pickle
fp = open("hsc_cosmos_35.sca", "wb")
pickle.dump(sources, fp)
fp.close()

The pickled/dilled file can be reopened in the same way. Every sources can be inspected as before. If we want to refit some data with this model, we have to recreate a `Blend` instances. 

In [ ]:
fp = open("hsc_cosmos_35.sca", "rb")
sources_ = pickle.load(fp)
fp.close()

blend_ = scarlet.Blend(sources_, observation)
model = blend_.get_model()
plt.imshow(scarlet.display.img_to_rgb(model, norm=norm))

We will now add two more sources to account for the largest residuals we have seen above. As we don't know their location accurately, we allow the fitter to shift/recenter the sources.

In [ ]:
# first freeze existing sources: they are not updated during fit
for src in sources_:
    src.freeze()

# add two sources at their approximate location
yx = (14., 44.)
new_source = scarlet.ExtendedSource(src.frame, yx, observation, shifting=True, symmetric=False, monotonic=True, thresh=5)
sources_.append(new_source)
yx = (43., 12.)
new_source = scarlet.ExtendedSource(src.frame, yx, observation, shifting=True, symmetric=False, monotonic=True, thresh=5)
sources_.append(new_source)

# generate a new Blend instance
blend_ = scarlet.Blend(sources_, observation)
# fit only new sources
blend_.fit(200)

# joint fit: fit all sources
blend_.unfreeze()
blend_.fit(200)

# show convergence of logL
print("scarlet ran for {0} iterations to logL = {1}".format(len(blend_.loss), -blend_.loss[-1]))
plt.plot(-np.array(blend.loss), 'k--', label='7 sources')
plt.plot(-np.array(blend_.loss), label='7+2 sources')
plt.xlabel('Iteration')
plt.ylabel('log-Likelihood')
plt.legend()

We can see that logL is slightly higher than before. To be clear, this is *not* the smartest way to refit a model. We did some fine-tuning with two new sources (for about 10 iterations), and then fit all of them again from scratch. That's where the wild oscillations come from and why it takes so long to converge. Anyway, let's have a look at the new model:

In [ ]:
# Load the model and calculate the residual
model = blend_.get_model()
model_ = observation.render(model)
residual = images-model_
# Create RGB images
model_rgb = scarlet.display.img_to_rgb(model_, norm=norm)
residual_rgb = scarlet.display.img_to_rgb(residual)

# Show the data, model, and residual
fig = plt.figure(figsize=(15,5))
ax = [fig.add_subplot(1,3,n+1) for n in range(3)]
ax[0].imshow(img_rgb)
ax[0].set_title("Data")
ax[1].imshow(model_rgb)
ax[1].set_title("Model")
ax[2].imshow(residual_rgb)
ax[2].set_title("Residual")

for k,component in enumerate(sources_):
    y,x = component.center
    ax[0].text(x, y, k, color="w")
    ax[1].text(x, y, k, color="w")
    ax[2].text(x, y, k, color="w")
plt.show()

We can see that most of the features are very similar to before. As expected, the residuals have visibly improved and are now dominated by a Yin-Yang-shaped orange-blue pattern in the center of source 1, which we already fit with 2 components. Maybe we should try with 3 ...